STOCK MARKET FORECASTING : ARIMA & PROPHET vs LSTM

This notebook compares simple statistical baselines (**ARIMA** and **Prophet**) with a lightweight **LSTM** on daily adjusted close prices for a few large-cap tickers (e.g., AAPL, TSLA, MSFT).  
We forecast **7 and 30 days** ahead, visualize results, and report RMSE/MAE on a recent holdout.

SETUP

In [18]:
import sys, subprocess
def pip(*args): subprocess.check_call([sys.executable, "-m", "pip", *args])

# 1) Clean out conflicting builds
pip("uninstall", "-y", "pmdarima", "prophet", "cmdstanpy", "pandas", "scikit-learn", "numpy")

# 2) Tooling
pip("install", "-U", "pip", "setuptools", "wheel", "cython")

# 3) Pin a consistent stack (works with TF 2.19 and Py3.12)
pip("install", "numpy==1.26.4")
pip("install", "pandas==2.1.4", "scikit-learn==1.4.2")

# 4) Reinstall packages that rely on NumPy ABI
#    Build pmdarima from source to guarantee it matches the current numpy headers.
pip("install", "--no-binary", "pmdarima", "pmdarima==2.0.4")

# Prophet stack
pip("install", "prophet==1.1.5", "cmdstanpy<1.2")


IMPORTS, BASELINES AND LSTM

In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt

import pmdarima as pm
from prophet import Prophet

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

pd.options.display.float_format = "{:,.4f}".format
print("NumPy:", np.__version__)
print("pmdarima:", pm.__version__)
print("TensorFlow:", tf.__version__)


NumPy: 1.26.4
pmdarima: 2.0.4
TensorFlow: 2.19.0


Config

In [20]:
TICKERS = ["AAPL","TSLA","MSFT"]   #
TEST_DAYS = 60                     # last 60 days as test window
FORECAST_WINDOWS = [7, 30]
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
def rmse(y_true, y_pred): return sqrt(mean_squared_error(y_true, y_pred)) #Root Mean Squared Error, measures how far predictions are from actuals, penalizing big mistakes more.
def mae(y_true, y_pred):  return mean_absolute_error(y_true, y_pred) #Mean Absolute Error, average size of errors

DATA LOADER

In [21]:
# load_series (keep)
def load_series(ticker, period_years=5):
    df = yf.download(ticker, period=f"{period_years}y", interval="1d", auto_adjust=True, progress=False)
    if df is None or df.empty: return pd.DataFrame(columns=["ds","y"])
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.get_level_values(0)
    price_col = "Close" if "Close" in df.columns else ("Adj Close" if "Adj Close" in df.columns else None)
    if price_col is None: return pd.DataFrame(columns=["ds","y"])
    out = df.dropna().rename(columns={price_col:"y"}).copy()
    if not isinstance(out.index, pd.DatetimeIndex):
        out.index = pd.to_datetime(out.index, errors="coerce")
    out["ds"] = out.index
    try: out["ds"] = out["ds"].dt.tz_localize(None)
    except: pass
    out = out[["ds","y"]].reset_index(drop=True)
    return _normalize_ds_y(out)


In [22]:
# _normalize_ds_y (keep)
def _normalize_ds_y(df):
    if not {"ds","y"}.issubset(df.columns):
        raise ValueError("Input DataFrame must contain 'ds' and 'y' columns.")
    out = df[["ds","y"]].copy()
    out["ds"] = pd.to_datetime(out["ds"], errors="coerce")
    out = out.dropna(subset=["ds"])
    y = out["y"]
    if y.apply(lambda v: isinstance(v, (list, tuple))).any():
        y = y.apply(lambda v: v[0] if isinstance(v, (list, tuple)) and len(v)>0 else None)
    y = pd.Series(y.values, index=out.index)
    y = pd.to_numeric(y, errors="coerce")
    out["y"] = y.astype("float64")
    out = out.dropna(subset=["y"]).sort_values("ds").reset_index(drop=True)
    try: out["ds"] = out["ds"].dt.tz_localize(None)
    except: pass
    return out


TRAIN/TEST SPLIT

In [23]:
def train_test_split(df, test_days=TEST_DAYS):
    return df.iloc[:-test_days].copy(), df.iloc[-test_days:].copy()

# quick smoke test
tmp = load_series("AAPL")
tmp.tail()

Price,ds,y
1251,2025-09-22,256.0800
1252,2025-09-23,254.4300
1253,2025-09-24,252.3100
1254,2025-09-25,256.8700
1255,2025-09-26,255.4600


In [24]:
def make_return_series(df):
    df = _normalize_ds_y(df).sort_values("ds")
    df["y"] = np.log(df["y"]).diff()   # log-returns
    df = df.dropna().reset_index(drop=True)
    return df


In [25]:
def prophet_forecast_on_dates(train_df, target_dates, cps=0.1, weekly=True):
    df = _normalize_ds_y(train_df)
    m = Prophet(
        daily_seasonality=False,
        weekly_seasonality=weekly,
        yearly_seasonality=False,
        changepoint_prior_scale=cps,
        changepoint_range=0.9,
        seasonality_mode="additive",
    )
    m.fit(df[["ds","y"]])
    future = pd.DataFrame({"ds": pd.to_datetime(pd.Index(target_dates), errors="raise").tz_localize(None)})
    preds  = m.predict(future)[["ds","yhat","yhat_lower","yhat_upper"]]
    return preds, m


ARIMA

In [26]:
def arima_forecast(train_df, horizon):
    df = _normalize_ds_y(train_df).sort_values("ds").reset_index(drop=True)
    y  = df.set_index("ds")["y"]  # DatetimeIndex here

    model = pm.auto_arima(
        y, seasonal=False, stepwise=True,
        suppress_warnings=True, error_action="ignore"
    )
    preds, conf = model.predict(n_periods=horizon, return_conf_int=True)

    start = y.index[-1] + pd.offsets.BDay(1)
    idx   = pd.bdate_range(start=start, periods=horizon)

    fcst = pd.DataFrame({
        "ds": idx,
        "yhat": preds,
        "yhat_lower": conf[:,0],
        "yhat_upper": conf[:,1],
    })
    return fcst, model


LSTM helper: create sequences

In [27]:
def lstm_forecast(train_df, horizon, lookback=60, epochs=20, batch=32, seed=42):
    df = _normalize_ds_y(train_df)
    vals = df["y"].values.astype("float32").reshape(-1,1)
    scaler = MinMaxScaler(); vals_sc = scaler.fit_transform(vals)
    X, y = [], []
    for i in range(lookback, len(vals_sc)):
        X.append(vals_sc[i-lookback:i, 0]); y.append(vals_sc[i, 0])
    if not X:  # too-short history
        return pd.DataFrame(columns=["ds","yhat","yhat_lower","yhat_upper"]), None
    X = np.array(X)[..., None]; y = np.array(y)
    tf.keras.utils.set_random_seed(seed)
    model = keras.Sequential([
        layers.Input(shape=(lookback,1)),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32),
        layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    model.fit(X, y, epochs=epochs, batch_size=batch, verbose=0, validation_split=0.1, callbacks=[cb])
    history = vals_sc.flatten().tolist(); preds_sc = []
    for _ in range(horizon):
        x = np.array(history[-lookback:], dtype="float32")[None, :, None]
        p = model.predict(x, verbose=0).ravel()[0]
        preds_sc.append(float(p)); history.append(float(p))
    preds = scaler.inverse_transform(np.array(preds_sc).reshape(-1,1)).ravel()
    start = pd.Timestamp(df["ds"].iloc[-1]) + pd.offsets.BDay(1)
    idx   = pd.bdate_range(start=start, periods=horizon)
    fcst = pd.DataFrame({"ds": idx, "yhat": preds, "yhat_lower": np.nan, "yhat_upper": np.nan})
    return fcst, model


In [28]:
# Baselines
def naive_forecast_on_dates(train_df, target_dates, method="last"):
    df = _normalize_ds_y(train_df).sort_values("ds")
    target_dates = pd.to_datetime(pd.Index(target_dates)).tz_localize(None)
    n = len(target_dates)
    last = df["y"].iloc[-1]
    if method == "last":
        yhat = np.full(n, last, dtype=float)
    elif method == "drift":
        y0, k = df["y"].iloc[0], max(1, len(df) - 1)
        drift = (last - y0) / k
        yhat = last + drift * np.arange(1, n + 1)
    else:
        raise ValueError("method must be 'last' or 'drift'")
    return pd.DataFrame({"ds": target_dates, "yhat": yhat, "yhat_lower": np.nan, "yhat_upper": np.nan})


EVALUATION_BACKTEST

In [29]:
def evaluate_models(df, horizon, test_days=60):
    df_xy   = _normalize_ds_y(df).sort_values("ds").reset_index(drop=True)
    train_df = df_xy.iloc[:-test_days].copy()
    test_df  = df_xy.iloc[-test_days:].copy()

    # ensure real tz-naive datetimes
    target_dates = pd.DatetimeIndex(pd.to_datetime(test_df["ds"].iloc[:horizon], errors="raise"))

    p_fcst, _ = prophet_forecast_on_dates(train_df, target_dates)
    a_fcst, _ = arima_forecast(train_df, horizon)
    l_fcst, _ = lstm_forecast(train_df, horizon)

    def coerce_to_target(fcst):
        if fcst is None or fcst.empty:
            return pd.DataFrame({"ds": target_dates, "yhat": np.nan,
                                 "yhat_lower": np.nan, "yhat_upper": np.nan})
        cols = ["ds","yhat"] + [c for c in ["yhat_lower","yhat_upper"] if c in fcst.columns]
        return pd.DataFrame({"ds": target_dates}).merge(fcst[cols], on="ds", how="left")

    a_fcst = coerce_to_target(a_fcst)
    l_fcst = coerce_to_target(l_fcst)

    n_last  = naive_forecast_on_dates(train_df, target_dates, "last")
    n_drift = naive_forecast_on_dates(train_df, target_dates, "drift")

    def align_and_score(name, fcst):
        merged = (fcst[["ds","yhat","yhat_lower","yhat_upper"]]
                  .merge(test_df[["ds","y"]].iloc[:horizon], on="ds", how="inner")
                  .sort_values("ds").reset_index(drop=True))
        before = len(merged)
        merged = merged.dropna(subset=["y","yhat"])
        if len(merged) < before:
            print(f"[warn] {name}: dropped {before-len(merged)} rows with NaNs.")
        if merged.empty:
            raise ValueError(f"{name}: no valid rows after alignment (NaNs).")
        return rmse(merged["y"].values, merged["yhat"].values), mae(merged["y"].values, merged["yhat"].values), merged

    scores, aligned = {}, {}
    for name, fc in [("Prophet", p_fcst), ("ARIMA", a_fcst), ("LSTM", l_fcst),
                     ("Naive-last", n_last), ("Naive-drift", n_drift)]:
        r, m, mdf = align_and_score(name, fc)
        scores[name]  = (float(r), float(m))
        aligned[name] = mdf
    return scores, aligned, test_df


In [30]:
print("train df check → using single brackets yields Series:",
      isinstance(pd.Series([1,2,3]), pd.Series))


train df check → using single brackets yields Series: True


PLOT HELPER

In [31]:
# Multi-model overlay plot
def plot_forecasts_multi(history_df, fcsts_dict, title="Forecast comparison"):
    """
    history_df: DataFrame with columns ['ds','y'] (e.g., your test_df or a tail of full history)
    fcsts_dict: {"Prophet": df, "ARIMA": df, "LSTM": df} each df must have ['ds','yhat'] and may have ['y']
                (evaluate_models returns merged dfs with both yhat and y)
    """
    fig = go.Figure()

    # 1) History
    hist = history_df.sort_values("ds")
    fig.add_trace(go.Scatter(x=hist["ds"], y=hist["y"], name="History", mode="lines"))

    # 2) Forecasts
    colors = {
        "Prophet": None,  # let Plotly pick; keeps style simple
        "ARIMA":   None,
        "LSTM":    None
    }
    for name, df in fcsts_dict.items():
        if df is None or df.empty:
            continue
        df = df.sort_values("ds")
        fig.add_trace(go.Scatter(
            x=df["ds"], y=df["yhat"], name=f"{name} forecast", mode="lines"
        ))

        # Confidence band if both bounds present and not all NaN
        if "yhat_lower" in df.columns and "yhat_upper" in df.columns:
            lower_ok = df["yhat_lower"].notna().any()
            upper_ok = df["yhat_upper"].notna().any()
            if lower_ok and upper_ok:
                fig.add_trace(go.Scatter(
                    x=pd.concat([df["ds"], df["ds"][::-1]]),
                    y=pd.concat([df["yhat_upper"], df["yhat_lower"][::-1]]),
                    fill="toself",
                    name=f"{name} CI",
                    line=dict(width=0),
                    opacity=0.15,
                    showlegend=True
                ))

        # If the merged frame includes actuals on those dates, show them
        if "y" in df.columns:
            fig.add_trace(go.Scatter(
                x=df["ds"], y=df["y"], name=f"Actual ({name} dates)",
                mode="markers", marker=dict(size=5), showlegend=False
            ))

    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Price (adj close)",
        hovermode="x unified"
    )
    fig.show()


In [32]:
t, H = TICKERS[0], 7
full = load_series(t)
train_df, test_df = train_test_split(full, test_days=TEST_DAYS)
target_dates = pd.to_datetime(test_df["ds"].iloc[:H],errors="raise")

for name, fn in [
    ("Prophet", lambda: prophet_forecast_on_dates(train_df, target_dates)),
    ("ARIMA",   lambda: arima_forecast(train_df, H)),
    ("LSTM",    lambda: lstm_forecast(train_df, H)),
]:
    try:
        _ = fn()
        print(f"{name}: OK")
    except Exception as e:
        print(f"{name}: FAIL → {e}")


DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/s78b6yz4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/htoplxjc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34086', 'data', 'file=/tmp/tmpakzlpboz/s78b6yz4.json', 'init=/tmp/tmpakzlpboz/htoplxjc.json', 'output', 'file=/tmp/tmpakzlpboz/prophet_modelpfiy8l2e/prophet_model-20250928223549.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:35:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:35:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet: OK


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



ARIMA: OK
LSTM: OK


RUN FOR YOUR TICKERS AND HORIZONS

In [33]:
# 10) Run for your tickers & horizons (multi-model overlay)
all_results = []
failed_runs = []

for t in TICKERS:
    df = load_series(t)  # already normalized to ['ds','y']
    if df is None or df.empty:
        print(f"Skipping {t}: no data returned.")
        failed_runs.append({"ticker": t, "horizon": None, "error": "no data"})
        continue

    print(f"\n=== {t} ===")
    for H in FORECAST_WINDOWS:
        try:
            scores, forecasts, test_df = evaluate_models(df, H)  # forecasts are aligned (ds,yhat[,y])
        except Exception as e:
            msg = f"evaluate_models failed for {t} @ H={H}d: {e}"
            print(msg)
            failed_runs.append({"ticker": t, "horizon": H, "error": str(e)})
            continue

        # Compact score line
        pretty = {k: (round(v[0], 3), round(v[1], 3)) for k, v in scores.items()}
        print(f"H={H}d  RMSE/MAE → {pretty}")

        # Overlay plot (don’t let plotting kill the run)
        try:
            plot_forecasts_multi(
                history_df=test_df,
                fcsts_dict=forecasts,
                title=f"{t} • {H}-day Forecasts (Prophet vs ARIMA vs LSTM)"
            )
        except Exception as e:
            print(f"[warn] plot failed for {t} @ H={H}d: {e}")

        # Log results one row per model
        for model_name, (rm, ma) in scores.items():
            all_results.append({
                "ticker": t,
                "horizon": H,
                "model": model_name,
                "rmse": float(rm),
                "mae": float(ma),
            })

# Results table (guarded)
scores_df = (pd.DataFrame(all_results)
             .sort_values(["ticker","horizon","rmse"])
             .reset_index(drop=True)
             if all_results else
             pd.DataFrame(columns=["ticker","horizon","model","rmse","mae"]))
scores_df

# Show failures (if any)
if failed_runs:
    print("\nFailures:")
    display(pd.DataFrame(failed_runs))
from IPython.display import display

# Make a scores dataframe you can sort/save
scores_df = pd.DataFrame(all_results)

print("rows: all_results =", len(all_results), " | failed_runs =", len(failed_runs))

if not scores_df.empty:
    scores_df = scores_df.sort_values(["ticker","horizon","rmse"]).reset_index(drop=True)
    display(scores_df.style.format({"rmse": "{:.3f}", "mae": "{:.3f}"}))
else:
    print("No model results to show.")
    if failed_runs:
        print("\nFailures:")
        display(pd.DataFrame(failed_runs))
    else:
        print("No failures either — likely the loop didn’t run or short-circuited.")



DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/th1ycbpf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/fwm40k7v.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=85138', 'data', 'file=/tmp/tmpakzlpboz/th1ycbpf.json', 'init=/tmp/tmpakzlpboz/fwm40k7v.json', 'output', 'file=/tmp/tmpakzlpboz/prophet_modelijr3oc6j/prophet_model-20250928223646.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:36:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing



=== AAPL ===


22:36:46 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



[warn] ARIMA: dropped 1 rows with NaNs.
[warn] LSTM: dropped 1 rows with NaNs.
H=7d  RMSE/MAE → {'Prophet': (21.16, 21.123), 'ARIMA': (1.665, 1.438), 'LSTM': (6.705, 6.435), 'Naive-last': (2.111, 1.778), 'Naive-drift': (2.412, 2.09)}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/o6wf3_i1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/gvgo9si8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=62274', 'data', 'file=/tmp/tmpakzlpboz/o6wf3_i1.json', 'init=/tmp/tmpakzlpboz/gvgo9si8.json', 'output', 'file=/tmp/tmpakzlpboz/prophet_modeloo_6cdua/prophet_model-20250928223749.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:37:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:37:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.12/dist-packag

...

[warn] ARIMA: dropped 1 rows with NaNs.
[warn] LSTM: dropped 1 rows with NaNs.
H=7d  RMSE/MAE → {'Prophet': (53.466, 53.424), 'ARIMA': (8.435, 8.094), 'LSTM': (20.786, 20.372), 'Naive-last': (9.903, 9.539), 'Naive-drift': (8.876, 8.57)}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/xnyy2n11.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpakzlpboz/m8n3j_jm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=7809', 'data', 'file=/tmp/tmpakzlpboz/xnyy2n11.json', 'init=/tmp/tmpakzlpboz/m8n3j_jm.json', 'output', 'file=/tmp/tmpakzlpboz/prophet_modelxoad8o69/prophet_model-20250928224154.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:41:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:41:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.12/dist-package

[warn] ARIMA: dropped 1 rows with NaNs.
[warn] LSTM: dropped 1 rows with NaNs.
H=30d  RMSE/MAE → {'Prophet': (63.269, 62.705), 'ARIMA': (20.265, 18.186), 'LSTM': (51.207, 47.199), 'Naive-last': (24.76, 22.363), 'Naive-drift': (20.609, 18.608)}


rows: all_results = 30  | failed_runs = 0


,ticker,horizon,model,rmse,mae
0,AAPL,7,ARIMA,1.665,1.438
1,AAPL,7,Naive-last,2.111,1.778
2,AAPL,7,Naive-drift,2.412,2.090
3,AAPL,7,LSTM,6.705,6.435
4,AAPL,7,Prophet,21.160,21.123
5,AAPL,30,ARIMA,7.650,5.059
6,AAPL,30,Naive-drift,7.876,5.472
7,AAPL,30,Naive-last,8.408,5.576
8,AAPL,30,LSTM,13.150,10.052
9,AAPL,30,Prophet,30.651,28.796


 Interpretation notes
- **Prophet**: captures weekly/yearly effects and tends to produce smoother trends; good uncertainty bands.
- **ARIMA**: often strong on short horizons; can struggle if regime shifts are frequent.
- **LSTM**: flexible but can overfit; our simple setup is for demonstration, not production.
- **Uncertainty**: We visualize bands for Prophet/ARIMA; LSTM here omits intervals for simplicity.
- **Evaluation**: We score on the last 60 trading days; for rigor, consider walk-forward CV (retrain each step).